In [1]:
import re
import numpy as np
import pandas as pd

# Función para detectar y sumar números en un rango específico en una cadena de texto
def sumar_numeros_rango(df, columna_texto):
    def extraer_y_sumar(text):
        # Encuentra todos los números en la cadena
        numeros = re.findall(r'\d+(?:\.\d{1,2})?', text)
        # Filtra los números dentro del rango 50-50000, conviértelos a float y sume
        numeros_en_rango = [float(num) for num in numeros if 50 <= float(num) <= 50000]
        # Si hay números en el rango, devuelve la suma; si no, devuelve NaN
        return sum(numeros_en_rango) if numeros_en_rango else np.nan

    # Aplicar la función a la columna especificada y guardar el resultado en una nueva columna 'Ingreso'
    df['Ingreso'] = df[columna_texto].apply(lambda x: extraer_y_sumar(str(x)))

data = pd.read_csv('data/REPARACIONES.csv', encoding='latin1')
data[['CODIGO', 'INFORMETALLER']]

,CODIGO,INFORMETALLER
0,1,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...
1,2,"FUENTE DAÑADA, FUSOR DAÑADO/INCOSTIABLE/ BASUR..."
2,2069,INCLUIR EL EDITOR DEL PDF $150+IVA
3,3,MANTENIMIENTO $480+IVA
4,28,"fusor completo. rodillo de calor dañado, funda..."
...,...,...
11945,11957,NaN
11946,11939,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...
11947,11940,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...
11948,11941,CAMBIO DE RODILLO DE CALOR $680 NETO\r\nMANTEN...


In [2]:
sumar_numeros_rango(data, 'INFORMETALLER')
data[['CODIGO', 'INFORMETALLER', 'Ingreso']]

,CODIGO,INFORMETALLER,Ingreso
0,1,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...,1820.0
1,2,"FUENTE DAÑADA, FUSOR DAÑADO/INCOSTIABLE/ BASUR...",NaN
2,2069,INCLUIR EL EDITOR DEL PDF $150+IVA,150.0
3,3,MANTENIMIENTO $480+IVA,480.0
4,28,"fusor completo. rodillo de calor dañado, funda...",NaN
...,...,...,...
11945,11957,NaN,NaN
11946,11939,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...,905.0
11947,11940,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...,2120.0
11948,11941,CAMBIO DE RODILLO DE CALOR $680 NETO\r\nMANTEN...,4218.0


In [3]:
data_cleaned = data.dropna(subset=['Ingreso'])
data_cleaned[['INFORMETALLER', 'Ingreso']]

,INFORMETALLER,Ingreso
0,CAMBIO DEL PANEL DE CONTROL $380+IVA BUEN MANT...,1820.0
2,INCLUIR EL EDITOR DEL PDF $150+IVA,150.0
3,MANTENIMIENTO $480+IVA,480.0
5,rep. de mylar de teclado $ 360.00 + iva,360.0
6,REPARACION DE MYLER DE TECLADO $ 360.00 + IVA,360.0
...,...,...
11941,*MANTENIMIENTO Y REPARACIÓN DE MEMBRANA DE TEC...,880.0
11944,RESORTE DE SENSOR DE PAPEL $400+IVA--ACEPTADO\...,3695.0
11946,MANTENIMIENTO EXHAUSTIVO $905 NETO \r\n\r\n\r\...,905.0
11947,*REQUIERE DE CAMBIO DE GOMA DE TRACCIÓN $380 N...,2120.0


In [4]:
data_cleaned['FALLA_STR'] = str(data_cleaned['FALLA'])
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9375 entries, 0 to 11948
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODIGO               9375 non-null   int64  
 1   CLIENTE              9375 non-null   int64  
 2   NS                   9374 non-null   object 
 3   FALLA                8959 non-null   object 
 4   OBSERVACIONES        4847 non-null   object 
 5   PROMETIDO            9375 non-null   object 
 6   FECHAINGRESO         9375 non-null   object 
 7   HORAINGRESO          9375 non-null   object 
 8   RECEPCIONADO         9375 non-null   int64  
 9   TECNICO              9375 non-null   int64  
 10  INFORMETALLER        9375 non-null   object 
 11  FECHATERMINADO       9375 non-null   object 
 12  HORATERMINADO        9172 non-null   object 
 13  ESTADO               9375 non-null   int64  
 14  ENTREGADO            9375 non-null   object 
 15  AVISADO              9375 non-null   

C:\Users\epile\AppData\Local\Temp\ipykernel_17516\1444229168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['FALLA_STR'] = str(data_cleaned['FALLA'])


In [5]:
import pandas as pd
import re
from collections import Counter

def limpiar_y_contar_fallas(df, columna_fallas):
    # Asegurarse de que la columna esté en formato de cadena, en minúsculas, y limpiar símbolos y números
    df[columna_fallas] = df[columna_fallas].astype(str).str.lower()
    df[columna_fallas] = df[columna_fallas].apply(lambda x: re.sub(r'[^a-záéíóúñü\s]', '', x))

    # Dividir cada cadena de fallas en palabras individuales
    fallas_separadas = df[columna_fallas].str.split()

    # Contar todas las ocurrencias de cada palabra (falla) en una lista plana
    contador_fallas = Counter([falla for lista_fallas in fallas_separadas for falla in lista_fallas])

    # Convertir el contador a un DataFrame ordenado por frecuencia de fallas
    df_fallas = pd.DataFrame(contador_fallas.items(), columns=['Falla', 'Frecuencia']).sort_values(by='Frecuencia', ascending=False)
    
    return df_fallas.reset_index(drop=True)


resultado_fallas = limpiar_y_contar_fallas(data, 'FALLA')
resultado_fallas.head(50)



,Falla,Frecuencia
0,no,4917
1,y,3486
2,se,2850
3,de,2813
4,la,2143
5,el,1906
6,las,1542
7,hojas,1372
8,imprime,1348
9,en,1290


In [6]:
len(resultado_fallas)

4790